# xcube resampling test - reprojection of an MSI scene to EPSG:4326

In [ ]:
import glob
import xarray as xr
import numpy as np
import dask.array as da
import pyproj
import matplotlib.pyplot as plt
from dask.distributed import Client
from xcube_resampling.grid import Axis, Grid
from xcube_resampling.reprojector import Reprojector

In [ ]:
%%time
#path = "../eopf_product_data_structure1/output/S2B_MSIL1C_20211122T043109_N0301_R133_T45QYD_20211122T063333.zarr/Measurements/Reflectances10m"
path = '/windows/tmp/eopf/S2B_MSIL1C_20211122T043109_N0301_R133_T45QYD_20211122T063333.zarr/Measurements/Reflectances10m'
l1c = xr.open_zarr(path, chunks={"row": 687*4, "column": 1373*4}, consolidated=False)

In [ ]:
l1c

We guess the source grid. It could be read from the input somehow.
We are south of Kalkutta.

In [ ]:
src_grid = Grid(pyproj.CRS("EPSG:32645"), Axis(2400000.0, -10.0, 10980), Axis(699960, 10.0, 10980), 2748, 5492)
target_crs = pyproj.CRS("EPSG:4326")
r = Reprojector(src_grid)
r.create_transformer(target_crs)
r.create_covering_target_grid(-0.0002, 0.0002, -0.4, 0.4)
print(r.target_grid)

In [ ]:
r.create_inverse_pixel_index()
repro_dask = r.reproject(l1c['B08'].data, l1c['B04'].data)

In [ ]:
repro_dask

In [ ]:
repro_dask.visualize()

The graph has 8 input chunks per band. There are 9 output chunks.

In [ ]:
%%time
repro_dask.compute()

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(5, 5))
plt.imshow(repro_dask.compute()[0], cmap='RdYlGn');

Distortion is small here.

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(5, 5))
plt.imshow(l1c['B08'], cmap='RdYlGn');